In [1]:
# Core imports 
from core.DecisionCell import DecisionCell
from core.Tooling import ToolHandler, ToolRegistry
from core.Agent import Agent

# Tooling imports
from tools.KnowledgeBaseTool import KnowledgeBaseTool
from tools.AskHumanTool import AskHumanTool

# DecisionCell  
A modular multi-agent framework for transparent, tool-augmented decision making

DecisionCell is a proof-of-concept multi-agent decision framework designed to support
collaborative reasoning through transparency and tool use.  
It enables multiple agents to think, act, call tools, and exchange observations
to collectively analyze a problem or objective.

The system combines:

- **ReAct-style reasoning** – agents think, take structured actions, call tools, and react to observations  
- **Transparent reasoning traces** – each agent exposes its step-by-step thought process  
- **Retrieval-Augmented Generation (RAG)** – agents can query their own knowledge bases to ground reasoning in data  

This notebook walks you through the full setup required to run a DecisionCell experiment.

## 1. Set up the staff (agents)

DecsionCell works with a small “staff” of collaborating agents. Each agent is defined by:

- A **role** (e.g. Commander, Intelligence, Logistics) encoded in a **system prompt**.
- A **provider** and **model** (e.g., Ollama (or HF) + Llama 3)
- Optional **tools** it can call (e.g. knowledge base, online search)

In the next cell we instantiate these agents and collect them in a staff list. 

### Provider Setup

You can run agents using either **Ollama (local models)** or **Hugging Face (cloud models)**.

#### **Using Ollama **  
Make sure the model is available and the service is running (see cell bellow)

#### **Using HF**  
Make sure to set your Hugging Face API key before running this notebook.

In [2]:
# Pull the models to be used for the agents
# !ollama pull qwen2.5:7b-instruct

# Confirm the ollama service is running
# !ollama list

In [3]:
# Build the staff list
staff = [
    # Agent("Commander", provider="ollama", model="qwen2.5:7b-instruct", debug=True),
    # Agent("LegalAdvisor", provider="ollama", model="qwen2.5:7b-instruct", debug=True),
    Agent("Commander", provider="hf", model="HuggingFaceH4/zephyr-7b-beta"),
    Agent("LegalAdvisor", provider="hf", model="HuggingFaceH4/zephyr-7b-beta"),
]

commander = staff[0]

Now that the agents are defined, we specify which tools they can use. Tools extend an agent’s capabilities beyond pure language modeling. For example, a tool may allow an agent to:

- Search its own knowledge base  
- Query external information  
- Ask a human for clarification  

Tools are declared in the `AGENT_TOOLS` list. By registering these tools with the `ToolRegistry`, we enable the agents to call them dynamically through structured actions.The `ToolHandler` is then responsible for routing each tool call to the correct tool implementation.

In [4]:
AGENT_TOOLS = [
    KnowledgeBaseTool(agent="LegalAdvisor", k=3, embedding_model = "normal", chunk_size=512, chunk_overlap=100),
    AskHumanTool("LegalAdvisor"), 
    AskHumanTool("Commander"),
]

registry = ToolRegistry(AGENT_TOOLS)
handler = ToolHandler(registry)

2026-01-11 13:08:35,401 - INFO - Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2026-01-11 13:08:43,856 - INFO - 1 prompt is loaded, with the key: query


## 2. Construct the DecisionCell (orchestrator)

The `DecisionCell` is the central orchestrator, it:

- Keeps track of the **conversation state** and the message stack.
- Decides **which agent acts next**.
- Routes **tool calls** through the `ToolHandler`.
- Stops when the objective is solved or the **max_turns** limit is reached.

In the next cell we create a `DecisionCell` by passing:
- The full `staff` list.
- The designated `commander` agent.
- The shared `tool_handler`.
- Some runtime settings such as `max_turns` and `debug`.

In [5]:
cell = DecisionCell(
    agents=staff,
    commander=commander,
    tool_handler=handler,
    max_turns=20,
    debug=True,
)

## 3. Define the mission objective

The mission objective describes the operational problem to be analyzed.

The objective is expressed as a multi-line natural-language description that
outlines the scenario and available Courses of Action (CoAs).  This text becomes 
the first input to the Commander when the decision process begins.

In [6]:
objective = """ 
Conduct a counter-terrorism cyber operation by a coalition of 12 countries to prevent an imminent attack by the Terrmisous group on a commercial cargo ship carrying chemical agents near the civilian port AricikPortus.

CoA 1: Disable the VicikPortus power grid using a previously implanted BlackEnergy3-based malware to prevent the terrorists from loading their cargo ship.
CoA 2: Temporarily neutralize the VicikPortus civilian pump station using a DDoS attack exploiting an unpatched vulnerability, preventing fueling of the terrorist vessel.
CoA 3: Conduct GNSS spoofing to alter the cargo ship's position, velocity, and heading after departure so it cannot reach its intended target.
"""

## 4. Run the decision cycle

With the staff, tools, and mission objective defined, we can now execute the full
decision cycle.  
The `DecisionCell` sends the objective to the Commander, who begins the analysis
and may consult other agents or call tools as needed.

During the cycle:

- Agents reason using the ReAct loop  
- Tool calls (e.g., knowledge base search or asking a human) are invoked when required  
- Observations are fed back into the agents' thinking  
- The process continues until a final recommendation is produced or the turn limit is reached  

The output includes:
- **RESULT** — the final recommendation from the Commander  
- **TRACE** — a step-by-step record of the agent actions and observations for transparency 

In [ ]:
logbook = cell.run(objective)

In [ ]:
# Text
text = logbook.console.export_text()

# HTML
html = logbook.console.export_html(inline_styles=True)

# Raw trace
trace = logbook.trace